In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.graphics.api import qqplot
sns.set(rc={'figure.figsize':(11.7,8.27)})

## Reading the datasets

df1 = pd.read_csv("data/BATADAL_dataset03.csv")
df2 = pd.read_csv("data/BATADAL_dataset04.csv")
dftest = pd.read_csv("data/BATADAL_test_dataset.csv")

In [4]:
# fitting a stepwise model:
from pmdarima.arima import auto_arima

stepwise_fit = auto_arima(wineind, start_p=1, start_q=1, max_p=3, max_q=3, m=12,
                          start_P=0, seasonal=True, d=1, D=1, trace=True,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True)  # set to stepwise

stepwise_fit.summary()



TypeError: unsupported operand type(s) for -: 'str' and 'str'

0     6.432172e-03
1     1.640237e-02
2     7.950662e-03
3     1.635197e-02
4     2.933532e-02
5     8.825696e-04
6     3.921161e-02
7     2.189767e-03
8     0.000000e+00
9     5.440216e-02
10    5.467896e-02
11             NaN
12             NaN
13    3.993735e-01
14    3.944880e-01
15             NaN
16             NaN
17    8.721980e-01
18    8.750000e-01
19    2.501832e-01
20    2.509444e-01
21    3.266815e-01
22    3.257866e-01
23             NaN
24             NaN
25    2.610466e-01
26    2.597220e-01
27    1.009522e+00
28    1.000000e+00
29    1.269635e-01
30    9.486537e-02
31    9.408749e-07
32    6.359138e-03
33    3.971544e-03
34    8.251070e-03
35    3.931639e-03
36    1.537696e-02
37    2.264306e-02
38    1.520302e-02
39    2.312966e-02
40    9.709491e-03
41    6.757152e-03
42    3.142531e-03
dtype: float64